In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
x1 = [0,0,1,1]
x2 = [0,1,0,1]
y = [0,1,1,0]

XOR_data = {'X1': x1,
            'X2' : x2,
           'y' : y }
df_data = pd.DataFrame(XOR_data)
df_data

,X1,X2,y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [4]:
X = df_data.drop("y",axis = 1)
Y = df_data['y']

In [5]:
X

,X1,X2
0,0,0
1,0,1
2,1,0
3,1,1


In [6]:
Y

,y
0,0
1,1
2,1
3,0


In [7]:
def Sigmoid(z):
  return 1 / (1 + np.exp(-z))

def ReQU(z):
  return np.square(z) * Sigmoid(z)

In [8]:
def Quantize(result):
  return 0 if result < 0.5 else 1

In [9]:
X.shape

(4, 2)

In [10]:
def forwardPropagation(X, parameters):
  hiddenNode01 = [] #ACT FUNC.
  hiddenNode02 = [] #ACT FUNC.
  NetworkOutputs = []
  QuantizedNetworkOutputs = []
  #To traverse through the data for 4 times, since X.shape --> (4,2) -->return 4
  for i in range(X.shape[0]):
    X_column = X[i,:].reshape((X.shape[1],1)) # -->(2,1)

    hiddenNode01_output = parameters['bias1'][0] + np.dot(X_column, parameters['weight1'][0])
    hiddenNode02_output = parameters['bias1'][1] + np.dot(X_column, parameters['weight1'][1])

    hiddenNode01.append(ReQU(hiddenNode01_output))
    hiddenNode02.append(ReQU(hiddenNode02_output))

    NetworkOutputs.append(Sigmoid(parameters['bias2'] + np.dot(parameters['weight2'][0],hiddenNode01[i]) + np.dot(parameters['weight2'][1]),hiddenNode02[i]))
    QuantizedNetworkOutputs.append(Quantize(NetworkOutputs[i]))

    return hiddenNode01, hiddenNode02, NetworkOutputs, QuantizedNetworkOutputs

In [11]:
def updateParameters(parameters, gradients, learningRate):

    parameters["weight1"][0][0] -= learningRate * gradients["w1_1_grad"]
    parameters["weight1"][0][1] -= learningRate * gradients["w2_1_grad"]
    parameters["weight1"][1][0] -= learningRate * gradients["w1_2_grad"]
    parameters["weight1"][1][1] -= learningRate * gradients["w2_2_grad"]
    parameters["weight2"][0] -= learningRate * gradients["w3_1_grad"]
    parameters["weight2"][1] -= learningRate * gradients["w3_2_grad"]
    parameters["bias1"][0] -= learningRate * gradients["bias_node1_grad"]
    parameters["bias1"][1] -= learningRate * gradients["bias_node2_grad"]
    parameters["bias2"][0] -= learningRate * gradients["biasForOutputNode_grad"]

    return parameters

In [12]:
def backpropagation(X1, X2, y, parameters, learningRate, epochCount):

  def ReQUGradient(z):
    z = np.asarray(z, dtype=np.float64)  # Z'yi float64'e çevir
    return Sigmoid(z) * (2 * z + np.square(z) * (1 - Sigmoid(z)))
  def SigmoidGradient(z):
    return Sigmoid(z) * (1 - Sigmoid(z))


  for epoch_iter in range(epochCount): # i.e. : 1000 over the whole dataset again...
    for iter_over_data in range(len(y)):

      #1st layer (also called: hidden layer) 1st Node weights
      w1_1 = parameters['weight1'][0][0]
      w1_2 = parameters['weight1'][0][1]

      #1st layer (also called: hidden layer) 2nd Node weights
      w2_1 = parameters['weight1'][1][0]
      w2_2 = parameters['weight1'][1][1]

      # 1st layer (also called: hidden layer) biases...
      bias1_1 = parameters['bias1'][0]
      bias1_2 = parameters['bias1'][1]

      #2nd layer (also called: output layer for this network) weights...
      w3_1 = parameters['weight2'][0]
      w3_2 = parameters['weight2'][1]

      #2nd layer (also called: output layer for this network) bias...
      biasForOutputNode = parameters['bias2']

      neuron1 = ReQU(bias1_1 + w1_1 * X1[iter_over_data] + w1_2 * X2[iter_over_data])
      neuron2 = ReQU(bias1_2 + w2_1 * X1[iter_over_data] + w2_2 * X2[iter_over_data])
      outputNeuron = Sigmoid(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2)


      #actual output:
      y_actual = y[iter_over_data]

      #Taking the gradient of output layer weights...
      w3_1_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * neuron1
      w3_2_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * neuron2

      #Taking the gradient of output layer bias...
      biasForOutputNode_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2)

      #Taking the gradient of hidden layer weights...
      w1_1_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_1 * ReQUGradient(bias1_1 + w1_1 * X1[iter_over_data] + w1_2 * X2[iter_over_data]) * X1[iter_over_data]
      w1_2_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_1 * ReQUGradient(bias1_1 + w1_1 * X1[iter_over_data] + w1_2 * X2[iter_over_data]) * X2[iter_over_data]

      w2_1_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_2 * ReQUGradient(bias1_2 + w2_1 * X1[iter_over_data] + w2_2 * X2[iter_over_data]) * X1[iter_over_data]
      w2_2_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_2 * ReQUGradient(bias1_2 + w2_1 * X1[iter_over_data] + w2_2 * X2[iter_over_data]) * X2[iter_over_data]

      #Taking the gradient of hidden layer biases...
      bias_node1_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_1 * ReQUGradient(bias1_1 + w1_1 * X1[iter_over_data] + w1_2 * X2[iter_over_data])
      bias_node2_grad = (outputNeuron - y_actual) * SigmoidGradient(biasForOutputNode + neuron1 * w3_1 + neuron2 * w3_2) * w3_2 * ReQUGradient(bias1_2 + w2_1 * X1[iter_over_data] + w2_2 * X2[iter_over_data])

      all_network_gradients = {'biasForOutputNode_grad' :biasForOutputNode_grad,
                               'w3_1_grad' : w3_1_grad,
                               'w3_2_grad' : w3_2_grad,
                               'bias_node1_grad' : bias_node1_grad,
                               'bias_node2_grad' : bias_node2_grad,
                               'w1_1_grad' : w1_1_grad,
                               'w1_2_grad' : w1_2_grad,
                               'w2_1_grad' : w2_1_grad,
                               'w2_2_grad' : w2_2_grad}
      parameters = updateParameters(parameters, gradients = all_network_gradients,learningRate = learningRate)
  return parameters

In [13]:
def initializeHandDeterminedParameters():
    parameters = {
                  "weight1": np.array([[1.36682195,1.43391771],[2.07983928,2.25466929]]),
                  "bias1": np.array([0.70575418,-1.68982321]),
                  "weight2": np.array([1.26826392,-2.7035043]),
                  "bias2" : np.array([-1.38142201])
                    }

    return parameters

In [14]:
parameters = initializeHandDeterminedParameters()
parameters

{'weight1': array([[1.36682195, 1.43391771],
        [2.07983928, 2.25466929]]),
 'bias1': array([ 0.70575418, -1.68982321]),
 'weight2': array([ 1.26826392, -2.7035043 ]),
 'bias2': array([-1.38142201])}

In [15]:
import random
def initializeRandomParameters():
    parameters = {
                  "weight1": np.array([[random.gauss(0, 1), random.gauss(0, 1)],[random.gauss(0, 1), random.gauss(0, 1)]]),
                  "bias1": np.array([random.gauss(0, 1), random.gauss(0, 1)]),
                  "weight2": np.array([random.gauss(0, 1), random.gauss(0, 1)]),
                  "bias2" : np.array([random.gauss(0, 1)])
                    }

    return parameters

In [16]:
initialParameters = initializeRandomParameters()

initialParameters

{'weight1': array([[ 0.61914117, -0.05239606],
        [-0.33739586, -0.986228  ]]),
 'bias1': array([0.68672224, 0.7342956 ]),
 'weight2': array([ 1.42545763, -0.26066236]),
 'bias2': array([0.41660052])}

In [17]:
learningRate = 0.01
epochCount = 10000

learnedParameters = backpropagation(x1, x2, y, initialParameters, learningRate, epochCount)
learnedParameters

{'weight1': array([[-1.78885335,  3.50896894],
        [-4.02210205, -1.70028327]]),
 'bias1': array([-1.46313578,  3.47387239]),
 'weight2': array([ 3.07363528, -2.89813015]),
 'bias2': array([0.19025399])}